In [3]:
import pickle as pkl
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


with open('/content/drive/MyDrive/Dataset_without_memory/dataset.pickle', 'rb') as f:
    dataset = pkl.load(f)

with open('/content/drive/MyDrive/Dataset_without_memory/labels.pickle', 'rb') as f:
    labels = pkl.load(f)

dataset.shape , labels.shape

labels = labels['1m class']
dataset = dataset.iloc[: , 1:]

import numpy as np
index1 = int(len(dataset) * 5/9)
index2 = int (len(dataset) * 6/9)


dataset_train , y_train  = np.asarray(dataset[: index1]).astype('float32') , np.asarray(labels[:index1 ]).astype('float32')

dataset_val , y_val = np.asarray(dataset[index1 : index2 ]).astype('float32') , np.asarray(labels[index1 : index2]).astype('float32')

dataset_test , y_test = np.asarray(dataset[index2 :  ]).astype('float32') , np.asarray(labels[index2:]).astype('float32')



def standardization( dataframe1, dataframe2, dataframe3  ):
  ###we start the for loop form the second column because the first column is timesteps 
  for i in range (dataframe1.shape[1]):
    
    retrn_data = dataframe1[:, i].reshape(-1, 1)

    scaler = StandardScaler()
    retrn_data_standardized = scaler.fit_transform(retrn_data)

    dataframe1[: , i] = retrn_data_standardized.flatten()


##########validation data
    retrn_data = dataframe2[:, i].reshape(-1, 1)

    retrn_data_standardized = scaler.transform(retrn_data)

    dataframe2[: , i] = retrn_data_standardized.flatten()

##########test data

    retrn_data = dataframe3[:, i].reshape(-1, 1)

    retrn_data_standardized = scaler.transform(retrn_data)

    dataframe3[: , i] = retrn_data_standardized.flatten()




  return dataframe1 , dataframe2, dataframe3

X_train , X_val , X_test = standardization(dataset_train , dataset_val , dataset_test)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import required libraries
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define model parameters
params = {
    'max_depth': 30,
    'eta': 0.5,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'lambda' : 0.5,
    'alpha' : 0.5

}

# Create DMatrix for training, validation and testing data
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test)

# Train the model
model = xgb.train(params, dtrain, num_boost_round=1000, early_stopping_rounds=12, evals=[(dval, 'validation')])

# Make predictions on the test data
y_pred = model.predict(dtest)

# Convert predictions to binary labels
y_pred_labels = [1 if p >= 0.5 else 0 for p in y_pred]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels)
recall = recall_score(y_test, y_pred_labels)
f1 = f1_score(y_test, y_pred_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


[0]	validation-logloss:0.74544
[1]	validation-logloss:0.77961
[2]	validation-logloss:0.80238
[3]	validation-logloss:0.82080
[4]	validation-logloss:0.83582
[5]	validation-logloss:0.84859
[6]	validation-logloss:0.85888
[7]	validation-logloss:0.86850
[8]	validation-logloss:0.87532
[9]	validation-logloss:0.88241
[10]	validation-logloss:0.88592
[11]	validation-logloss:0.89333
Accuracy: 0.5016
Precision: 0.4875
Recall: 0.6755
F1-score: 0.5663
